In [1]:
# Import libraries
import requests
import json
from config import api_key
from pprint import pprint
import time
from random import randint

In [2]:
# Test API key
api_key

'BQDVqlFY8HwQdJqs8LpIukcCKJbNTZZmasXQRsvxqv1z9a_Yd79xq4dgJbiRS96BdQ3k2_Y0YKHfkyQoLDHJUXxLaPDAfHA3GazLyO4be5yh5lVS9v6ionGMjCZA7U9Y8QKREdsdqVAQyYm967GF'

SPOTIFY DOCUMENTATION
https://developer.spotify.com/console/tracks/

Example: wk06, 02, 07

In [3]:
#[create list of songs from database to query]
import pandas as pd
import sqlite3

songs_csv = "../../Resources/spotify_tracks_features.csv"
df = pd.read_csv(songs_csv)

# Clean columns for initial pass of CSV
# df.columns = df.columns.str.strip()
# df.columns = df.columns.str.replace(".","")
df.head()

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],1,1,False,0.470,...,0.0727,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,1999,1999-11-02
1,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],2,1,True,0.599,...,0.1880,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,1999,1999-11-02
2,1hR0fIFK2qRG3f3RF70pb7,Calm Like a Bomb,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],3,1,False,0.315,...,0.4830,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,1999,1999-11-02
3,2lbASgTSoDO7MTuLAXlTW0,Mic Check,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],4,1,True,0.440,...,0.2370,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,1999,1999-11-02
4,1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],5,1,False,0.426,...,0.0701,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,1999,1999-11-02


In [4]:
# Create sequential batches of track IDs to query with API

# df_4000 = df.loc[0:4000]
# song_ids_1 = df_4000['id'].tolist()

# df_8000 = df.loc[4001:8000]
# song_ids_2 = df_8000['id'].tolist()

# df_12000 = df.loc[8001:12000]
# song_ids_3 = df_12000['id'].tolist()

# df_16000 = df.loc[12001:16000]
# song_ids_4 = df_16000['id'].tolist()

# df_20000 = df.loc[16001:20000]
# song_ids_5 = df_20000['id'].tolist()

# df_24000 = df.loc[26885:]
# song_ids_6 = df_24000['id'].tolist()

# df_47000 = df.loc[37577:47000]
# song_ids_7 = df_47000['id'].tolist()

# df_57000 = df.loc[47001:57000]
# song_ids_8 = df_57000['id'].tolist()

df_77000 = df.loc[57001:77000]
song_ids_10 = df_77000['id'].tolist()

In [4]:
# Create song list of songs without 'popularity'

df_has_rating = df.dropna(subset=['popularity'])
df_no_pop_rating = df.drop(df_has_rating.index)
song_ids = df_no_pop_rating['id'].tolist()

song_ids

['3v3ak1HPkCorepvdpSWDLw',
 '2T81p9MeYDvN3g0BObGuCa',
 '5E4kJCuitB4BiDgz0Mx7Gp',
 '1GJLq5qDfVHCSvtGlCzqMz',
 '65X7vFxMzt2QKZsnirzCvl',
 '2pX2KL8DuL5A9piw1gJBR8',
 '7xlt9ruej6HNWFp78wA2hx',
 '7J9vinRXin3btUODPXWAKI',
 '2uCtWqIvKGkSAmjvCcazog',
 '6OWzZzAdLat8y3DiL8Io9E',
 '7GGThL3iDBBp221rvCiAJN',
 '4I0m4m2dHoDZqv9aqp83sV',
 '3QvH4oX3XMNR5JbtU3d2S8',
 '1OEtFC67n26vElDgqXHrow',
 '0jIz8A2HkEvbz1AGuJDA6E',
 '5WoLVMFMa6BnDnUzj5w7KQ',
 '2I7Y4H982U7DXmYzFgVRsL',
 '2mj6nSqwd0ukjnfTlRded9',
 '3vh1fYct19MPmRZuU9jysO',
 '1PLVQbGwtyZHBWfGmYh3AS',
 '6Cse8eqVxkqCWJRx9XrZCK',
 '4yDtFhSJ4zVgv3juc49s5p',
 '7hGbvpUc7NeKVuLhzbp9aF',
 '0azpYwyWUUhipeGLFtsmom',
 '1WCiRpGMdAemJohMgteHMK',
 '5hXSpjvHO2vQKAR8TZCMyv',
 '2yUKEdoQYDZpuNrEA1Jjou',
 '6d8QeZpIvoNy0Fdz7H4jaN',
 '25UJi2iP2qeBIu8OPVkM8Y',
 '1RkIxY6ciLLjlxbeNim0Jd',
 '2Bq0tAKMYk7kUvsVKzqy9Q',
 '1LrkHmF22aIVLFB9X7XbSo',
 '4A6xj1Yr1AbIC3k886RzeG',
 '37byw8uDWSjaarMR3tVifQ',
 '5G53CaolWC4xGnTnwV8pzU',
 '39SUpkJ7V6OhUWeoTBFIdV',
 '3qWbHgl9qg3HUd78mAueis',
 

In [ ]:
# FOR CHECKING WHETHER API KEY IS STILL WORKING

import requests
from requests.structures import CaseInsensitiveDict
import numpy as np

url = "https://api.spotify.com/v1/tracks/3JbxyXrJDf4tNLvIQI2iXl"
headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"
headers["Authorization"] = f"Bearer {api_key}"

response = requests.get(url, headers=headers).json()

response

In [5]:
import requests
from requests.structures import CaseInsensitiveDict
import numpy as np

# set up lists to hold reponse info
popularity = []
# Add dictionary to list

url = "https://api.spotify.com/v1/tracks/"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"
headers["Authorization"] = f"Bearer {api_key}"

# Loop through the list of songs and perform a request for data on each
for song in song_ids_10:
    # resp = requests.get(query_url, headers=headers)
    try:
        response = requests.get(url + song, headers=headers).json()
        adict = {'id':song,'popularity':response['popularity']}
        popularity.append(adict)
        # print(f"Song popularity for {song}: {response['popularity']}")
        print(adict)
    except:
        time.sleep(31)
        try:
            response = requests.get(url + song, headers=headers).json()
            adict = {'id':song,'popularity':response['popularity']}
            popularity.append(adict)
            # print(f"Song popularity for {song}: {response['popularity']}")
            print(adict)
        except:
            adict = {song:(np.nan)}
            popularity.append(adict)
            print(response.status_code)

        # print(f"Song popularity for {popularity}:")

    # time.sleep(randint(1,5))

# print(resp.status_code)
# resp.json()
# print(response['popularity'])
# print(api_key)

# pandas.dataframe.append
# popularity.append(response['popularity'])
# popularity.append(np.nan)

{'id': '3ImutPiSzr0QI68peEUYwE', 'popularity': 4}
{'id': '4sTxfmHToppzkVKsDTsJYM', 'popularity': 4}
{'id': '5BcMeqqAffh5itmQvhCIMe', 'popularity': 3}
{'id': '1Rjuq02eFqvFiLCZ6Eaq5u', 'popularity': 3}
{'id': '021thINqQv0Xi6z3u3qNKA', 'popularity': 3}
{'id': '72pBrhQAbiSxqyJ9wauHiN', 'popularity': 4}
{'id': '6mmbkkKrAWDbOMF5pwiecE', 'popularity': 3}
{'id': '331PPPtN1z86N4ZaSKHtvG', 'popularity': 2}
{'id': '3dI8fayyUR4QlzI45GhzlV', 'popularity': 4}
{'id': '4E9WJVrOkiRV1xrYT1uT41', 'popularity': 4}
{'id': '6F89Er0aazpJpYiXFq4Plw', 'popularity': 4}
{'id': '3UAS1yg5XyZfV763vjngdA', 'popularity': 4}
{'id': '5Ul5mhhFkh3NWtP6VRnxLn', 'popularity': 4}
{'id': '7nIqWqZ3WPygE1n0vSDlVK', 'popularity': 11}
{'id': '0yizwQL4cadhP2wQSZ3oNO', 'popularity': 6}
{'id': '3HnNpCEpYYowyxwXdnfCGL', 'popularity': 4}
{'id': '4fneBxlattRkyD38HCIphh', 'popularity': 5}
{'id': '17mKkvM7jWkDvvjrwUavxk', 'popularity': 6}
{'id': '2hGEM8fk2cwKlLnvJbnng4', 'popularity': 4}
{'id': '6HS2qNJiXIkQ9EJaMN1QkE', 'popularity': 3}

AttributeError: 'dict' object has no attribute 'status_code'

In [9]:
# df_popularity = pd.DataFrame.from_records(popularity)
df_popularity_77000 = pd.DataFrame.from_dict(popularity)
# df_popularity_77000 = df_popularity_77000.dropna(how='all') #Drop all rows with NaN
df_popularity_77000 = df_popularity_77000.drop('2Khb67HryFTtRWsDY4siXC',1) #Drop final column
df_popularity_77000.sort_values(by='popularity',ascending=False)
# df_popularity_57000.columns

,id,popularity
10862,3ZFTkvIE7kyPt6Nu3PEa7V,82.0
49,1ixbwbeBi5ufN4noUKmW5a,77.0
10002,0oxYB9GoOIDrdzniNdKC44,65.0
8650,7nVQ8mo77KaUvhUQzh4vMy,65.0
8485,1qOU8KzFifXE9YrgjVwYvc,64.0
...,...,...
7388,6Ha0zPCMyn3cXvi9JYdCJH,0.0
7387,76Eqap2UPu650j5Ep9ndei,0.0
7386,3wlEoDXKB8ouvqpmM6EvqA,0.0
5538,25ZtSjLdfFfogYhLM9zXWQ,0.0


In [10]:

merged_df_10 = pd.merge(df, df_popularity_77000, how='right')
merged_df_10.head()

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date,popularity
0,3ImutPiSzr0QI68peEUYwE,"All I Want Is You - Live in Cologne, Germany, ...",Unzipped (Super Deluxe Edition),2nLoLhMXyu6Fa1W6QLmiFF,['Whitesnake'],['3UbyYnvNIT5DFXU4WgiGpP'],15.0,3.0,False,0.363,...,0.716,0.000034,0.202,0.412,81.158,309347.0,1.0,2018.0,2018-10-19,4.0
1,4sTxfmHToppzkVKsDTsJYM,"Slow an' Easy - Live in Cologne, Germany, 24/1...",Unzipped (Super Deluxe Edition),2nLoLhMXyu6Fa1W6QLmiFF,['Whitesnake'],['3UbyYnvNIT5DFXU4WgiGpP'],16.0,3.0,False,0.340,...,0.588,0.000006,0.684,0.623,89.366,266893.0,4.0,2018.0,2018-10-19,4.0
2,5BcMeqqAffh5itmQvhCIMe,"Dog - Live in Cologne, Germany, 24/11/2006; 20...",Unzipped (Super Deluxe Edition),2nLoLhMXyu6Fa1W6QLmiFF,['Whitesnake'],['3UbyYnvNIT5DFXU4WgiGpP'],17.0,3.0,False,0.420,...,0.746,0.002020,0.794,0.879,107.899,149667.0,4.0,2018.0,2018-10-19,3.0
3,1Rjuq02eFqvFiLCZ6Eaq5u,"Give Me All Your Love - Live in Paris, France,...",Unzipped (Super Deluxe Edition),2nLoLhMXyu6Fa1W6QLmiFF,['Whitesnake'],['3UbyYnvNIT5DFXU4WgiGpP'],18.0,3.0,False,0.449,...,0.702,0.000002,0.971,0.547,133.526,164867.0,4.0,2018.0,2018-10-19,3.0
4,021thINqQv0Xi6z3u3qNKA,"Ain't Gonna Cry No More - Live in Paris, Franc...",Unzipped (Super Deluxe Edition),2nLoLhMXyu6Fa1W6QLmiFF,['Whitesnake'],['3UbyYnvNIT5DFXU4WgiGpP'],19.0,3.0,False,0.463,...,0.809,0.000002,0.789,0.435,101.711,176693.0,4.0,2018.0,2018-10-19,3.0


In [11]:
merged_df_10.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11078 entries, 0 to 11077
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                11077 non-null  object 
 1   name              11077 non-null  object 
 2   album             11077 non-null  object 
 3   album_id          11077 non-null  object 
 4   artists           11077 non-null  object 
 5   artist_ids        11077 non-null  object 
 6   track_number      11077 non-null  float64
 7   disc_number       11077 non-null  float64
 8   explicit          11077 non-null  object 
 9   danceability      11077 non-null  float64
 10  energy            11077 non-null  float64
 11  key               11077 non-null  float64
 12  loudness          11077 non-null  float64
 13  mode              11077 non-null  float64
 14  speechiness       11077 non-null  float64
 15  acousticness      11077 non-null  float64
 16  instrumentalness  11077 non-null  float6

In [12]:
merged_df_10.to_csv('../../Resources/merged_df_10.csv')

In [24]:
# Save to csv
df_popularity = pd.DataFrame(popularity, columns=['popularity'])
df_popularity.to_csv('../../Resources/popularity_2.csv')

In [ ]:
# Concat to popularity csv